In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
from tqdm.notebook import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [35]:
train_df=pd.read_csv("../input/businessandai-competition/Xy_train.csv", sep=";")
test_df=pd.read_csv("../input/businessandai-competition/X_test.csv", sep=";")
sub=pd.read_csv("../input/businessandai-competition/submission_sample.csv", sep=";")

In [36]:
train_df.head()

,_id,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,num_5,categ_8,...,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,num_6,num_2,num_7,y
0,625267cf0a63071b21c47e2b,2,"a:o@`''E]@W@`'']@eBcqK~@mo""c<@1!)@`RRK@Kw",4,168000,1,0,49,NaN,4,...,5,2,<o<,2008.0,2368,<o<,NaN,NaN,NaN,4500.0
1,625666e07a2d1ced83779994,2,"a:o@eKc<:`@Z!)@K@""]:@R:e:c<@Oq'Yeoee@+:`DKoee@...",4,50,3,0,50,115.0,41,...,5,2,"gEO+3'KccKDEb@Eq'""3+oK'3q@]'@R:K'ee'Eb@Eqo]oq@...",2022.0,2368,<o<,NaN,998.0,NaN,22133.0
2,61b9ed1f7e264de96093187b,1,"a:o@eKc<:`@+o3<`D@']:K:c<@Z!)@Kd""]:@Z1)",4,85717,3,0,34,115.0,41,...,1,2,g}`oY'qo@]Eo:]'@o3@eKoK:c<<'Y'<K}b@}`oBK'3qe@]...,2017.0,2296,"R'D:`3+'@`c<Kqc+'@d@""oqo<K:'@;@Yc:e@:<`+3e'@G'...",NaN,NaN,6.0,11490.0
3,624ac4947e9fdee78b7eba63,2,"a:o@q:c@Z!)@K@""]:@Z1)@B+oK:<3Y@']:K:c<",4,30000,3,0,49,NaN,33,...,10,1,<o<,2017.0,2368,<o<,NaN,NaN,NaN,14500.0
4,6254d841bef56d2c1cf934ea,2,"a:o@eBcqKo""'@hoo@e'q:'",2,173000,3,0,49,NaN,39,...,8,1,<o<,2013.0,2368,<o<,NaN,NaN,NaN,10000.0


In [37]:
test_df.head()

,_id,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,num_5,categ_8,...,num_10,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,num_6,num_2,num_7
0,6251210db22e614680cef17b,2,a:o@`oq'<e@@@1!)@`RRK@w@Kqo@o:q`c,4,168982,3,0,49,NaN,1,...,20.0,11,2,<o<,2007.0,2368,<o<,NaN,NaN,NaN
1,609a41b746383f48ea7ed9c4,2,"a:o@eBcqKo""'@Z!?@`q]:@ZZu@:e""@,w1@o`K:R'",2,74500,3,0,19,119.0,39,...,999.0,1,1,gEo:q@`c<]:K:c<<'Eb@E`oY'qo@]'@q'`3+Eb@EO+3'Kc...,NaN,2368,<o<,NaN,NaN,NaN
2,609b46c346383f48ea85a360,2,"a:o@q:c@Bq'Y:3Y@Z!,+@Z)_",4,82996,3,0,34,114.0,33,...,999.0,1,1,<o<,NaN,2368,<o<,NaN,NaN,NaN
3,609a42ee46383f48ea7ef087,2,a:o@B:`o<Kc@Z!)@;;`D@o`K:R'@uB@`+:Y@Yo<3@3eO@O...,4,36470,3,0,19,105.0,25,...,999.0,1,1,gEO+3'KccKDEb@Ecq]:<oK'3q@]'@Ocq]Es,2016.0,2368,<o<,NaN,NaN,NaN
4,609a42cd46383f48ea7eed93,2,"a:o@eBcqKo""'@1!)@`q]:@Z,)@,w1@.oB@3qOo<@q:]'q",2,185000,3,0,51,184.0,39,...,999.0,1,1,gEo:q@`c<]:K:c<<'Es,NaN,2368,<o<,NaN,NaN,NaN


In [38]:
test_df.shape

(15000, 27)

In [39]:
Id=test_df["_id"]

In [40]:
train_df.shape

(45000, 28)

In [41]:
train_df["y"].nunique()

5457

In [42]:
train_df.isna().sum().sort_values(ascending =False).head(20)

num_6      43318
num_7      42688
num_9      41739
num_8      35759
num_2      35674
num_5      32416
num_3      30466
num_1      24203
num_4       5276
num_10      1536
y              1
txt_1          0
categ_3        0
txt_3          0
categ_7        0
categ_6        0
_id            0
categ_9        0
date_0         0
categ_0        0
dtype: int64

In [43]:
train_df.isna().sum().sort_values(ascending =False).head(20)/len(train_df)

num_6      0.962622
num_7      0.948622
num_9      0.927533
num_8      0.794644
num_2      0.792756
num_5      0.720356
num_3      0.677022
num_1      0.537844
num_4      0.117244
num_10     0.034133
y          0.000022
txt_1      0.000000
categ_3    0.000000
txt_3      0.000000
categ_7    0.000000
categ_6    0.000000
_id        0.000000
categ_9    0.000000
date_0     0.000000
categ_0    0.000000
dtype: float64

In [44]:
train_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1"], axis=1, inplace=True)
test_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1"], axis=1, inplace=True)


In [45]:
train_df.shape

(45000, 19)

In [46]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   categ_0  45000 non-null  int64  
 1   txt_0    45000 non-null  object 
 2   categ_1  45000 non-null  int64  
 3   num_0    45000 non-null  int64  
 4   categ_2  45000 non-null  int64  
 5   categ_4  45000 non-null  int64  
 6   categ_5  45000 non-null  int64  
 7   categ_8  45000 non-null  int64  
 8   txt_2    45000 non-null  object 
 9   date_0   45000 non-null  object 
 10  categ_9  45000 non-null  int64  
 11  num_10   43464 non-null  float64
 12  categ_6  45000 non-null  int64  
 13  categ_7  45000 non-null  int64  
 14  txt_3    45000 non-null  object 
 15  num_4    39724 non-null  float64
 16  categ_3  45000 non-null  int64  
 17  txt_1    45000 non-null  object 
 18  y        44999 non-null  float64
dtypes: float64(3), int64(11), object(5)
memory usage: 6.5+ MB


In [47]:
import datetime as dt
train_df["date_0"]=pd.to_datetime(train_df["date_0"])
train_df['Date_year'] =train_df["date_0"].dt.year
train_df['Date_month'] = train_df["date_0"].dt.month
train_df['Date_day'] = train_df["date_0"].dt.day
train_df.drop("date_0", axis=1, inplace=True)

In [48]:
test_df["date_0"]=pd.to_datetime(test_df["date_0"])
test_df['Date_year'] =test_df["date_0"].dt.year
test_df['Date_month'] = test_df["date_0"].dt.month
test_df['Date_day'] = test_df["date_0"].dt.day
test_df.drop("date_0", axis=1, inplace=True)

In [49]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categ_0     45000 non-null  int64  
 1   txt_0       45000 non-null  object 
 2   categ_1     45000 non-null  int64  
 3   num_0       45000 non-null  int64  
 4   categ_2     45000 non-null  int64  
 5   categ_4     45000 non-null  int64  
 6   categ_5     45000 non-null  int64  
 7   categ_8     45000 non-null  int64  
 8   txt_2       45000 non-null  object 
 9   categ_9     45000 non-null  int64  
 10  num_10      43464 non-null  float64
 11  categ_6     45000 non-null  int64  
 12  categ_7     45000 non-null  int64  
 13  txt_3       45000 non-null  object 
 14  num_4       39724 non-null  float64
 15  categ_3     45000 non-null  int64  
 16  txt_1       45000 non-null  object 
 17  y           44999 non-null  float64
 18  Date_year   45000 non-null  int64  
 19  Date_month  45000 non-nul

In [50]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categ_0     15000 non-null  int64  
 1   txt_0       15000 non-null  object 
 2   categ_1     15000 non-null  int64  
 3   num_0       15000 non-null  int64  
 4   categ_2     15000 non-null  int64  
 5   categ_4     15000 non-null  int64  
 6   categ_5     15000 non-null  int64  
 7   categ_8     15000 non-null  int64  
 8   txt_2       15000 non-null  object 
 9   categ_9     15000 non-null  int64  
 10  num_10      14536 non-null  float64
 11  categ_6     15000 non-null  int64  
 12  categ_7     15000 non-null  int64  
 13  txt_3       15000 non-null  object 
 14  num_4       13237 non-null  float64
 15  categ_3     15000 non-null  int64  
 16  txt_1       15000 non-null  object 
 17  Date_year   15000 non-null  int64  
 18  Date_month  15000 non-null  int64  
 19  Date_day    15000 non-nul

# Dealing with text data

In [51]:
txt=pd.DataFrame({"txt_0":train_df["txt_0"],"txt_2":train_df["txt_2"] ,"txt_1":train_df["txt_1"] ,"txt_3":train_df["txt_3"]})

In [66]:
txtt=pd.DataFrame({"txt_0":test_df["txt_0"],"txt_2":test_df["txt_2"] ,"txt_1":test_df["txt_1"] ,"txt_3":test_df["txt_3"]})

In [52]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [53]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
# punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    return s

start=time.time()
for i in tqdm(range(txt.shape[0])):
    txt.loc[i,"txt_0"] = process(txt.loc[i,"txt_0"])
    txt.loc[i,"txt_1"] = process(txt.loc[i,"txt_1"])
    txt.loc[i,"txt_2"] = process(txt.loc[i,"txt_2"])
    txt.loc[i,"txt_3"] = process(txt.loc[i,"txt_3"])

for i in tqdm(range(txt.shape[0])):
    txt.loc[i,"txt_0"] = " ".join(txt.loc[i,"txt_0"])
    txt.loc[i,"txt_1"] = " ".join(txt.loc[i,"txt_1"])
    txt.loc[i,"txt_2"] = " ".join(txt.loc[i,"txt_2"])
    txt.loc[i,"txt_3"] = " ".join(txt.loc[i,"txt_3"])
end=time.time()
print(end-start)

  0%|          | 0/45000 [00:00<?, ?it/s]

  0%|          | 0/45000 [00:00<?, ?it/s]

109.98027873039246


In [ ]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
# punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    return s

start=time.time()
for i in tqdm(range(txt.shape[0])):
    txt.loc[i,"txt_0"] = process(txt.loc[i,"txt_0"])
    txt.loc[i,"txt_1"] = process(txt.loc[i,"txt_1"])
    txt.loc[i,"txt_2"] = process(txt.loc[i,"txt_2"])
    txt.loc[i,"txt_3"] = process(txt.loc[i,"txt_3"])

for i in tqdm(range(txt.shape[0])):
    txt.loc[i,"txt_0"] = " ".join(txt.loc[i,"txt_0"])
    txt.loc[i,"txt_1"] = " ".join(txt.loc[i,"txt_1"])
    txt.loc[i,"txt_2"] = " ".join(txt.loc[i,"txt_2"])
    txt.loc[i,"txt_3"] = " ".join(txt.loc[i,"txt_3"])
end=time.time()
print(end-start)

In [54]:
txt

,txt_0,txt_2,txt_1,txt_3
0,aoEWeBcqKmoc1RRKKw,o,o,o
1,aoeKcZKRecOqYeoeeDKoeeeKjD,gEO3KccKDEbEq3oK3qRKeeEbEqooqq3EbEeeDo3oKeEbEo...,o,gEO3KccKDEbEq3oK3qRKeeEbEqooqq3EbEeeDo3oKeEbEo...
2,aoeKco3DKcZKdZ1,goYqoEoo3eKoKcYKboBK3qeEoo3eKoKcYKoqqqbEq3oK3q...,RD3cKqcdoqoKYce3eGwKecBceeOdeoKeoKc3qYOc3qeBoK...,goYqoEoo3eKoKcYKboBK3qeEoo3eKoKcYKoqqqbEq3oK3q...
3,aoqcZKZ1BoK3YKc,o,o,o
4,aoeBcqKohooeq,o,o,o
...,...,...,...,...
44995,aoqcZDRoKRccqKKqoK,gEoqcKcEbEoYqoq3EbEBeEbEo3Yoo3KcYoKf3e3wEbEee3...,o,gEoqcKcEbEoYqoq3EbEBeEbEo3Yoo3KcYoKf3e3wEbEee3...
44996,aowZBDRKDo3K,o,o,o
44997,aoec3BqY3Y1D,gEoc3cqEbEoDoKKDo3KEbEoeeeKoKYoqqocKEboYqoEoo3...,OooqMZDo3KBoq3qeMoOeMoc3cqKqooRoRqoYKMoDoKKDo3...,gEoc3cqEbEoDoKKDo3KEbEoeeeKoKYoqqocKEboYqoEoo3...
44998,aoeBcqKoZqZZuRq,o,o,o


In [55]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(max_features=1000)
bow_data = vectorizer.fit_transform(txt["txt_0"]).toarray()
features = vectorizer.get_feature_names_out()
bow_data = pd.DataFrame(bow_data, columns=features)

In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(max_features=1000)
bow_data1 = vectorizer.fit_transform(txt["txt_1"]).toarray()
features = vectorizer.get_feature_names_out()
bow_data1 = pd.DataFrame(bow_data1, columns=features)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(max_features=1000)
bow_data2 = vectorizer.fit_transform(txt["txt_2"]).toarray()
features = vectorizer.get_feature_names_out()
bow_data2 = pd.DataFrame(bow_data2, columns=features)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(max_features=1000)
bow_data3 = vectorizer.fit_transform(txt["txt_3"]).toarray()
features = vectorizer.get_feature_names_out()
bow_data3 = pd.DataFrame(bow_data3, columns=features)

In [64]:
bow = pd.concat([bow_data, bow_data1,bow_data2,bow_data3], axis=1)

In [60]:
bow

,31,ao,ao1z,ao4wez4hvvqr,aoaobokc,aoaobokczokrubcqk,aoaoebcqko,aoaoebcqkozquamcc1m,aoaoebcqkozqzhrydoqo3ee,aoaoecqkc,...,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbeyokeokco3kcyokf3bodjcebekk3q3yqebekk3qb3ebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeekoqkwekcbo3kcyokf3ebeeekyorokceberkqekqf3eebercok3qebercoky3kckceebeebcekyoeoqeebecqok3qocqebeec3eekyebe3eoebe9okeooebeoqooc3k3qebeoqooboeeoqebeoqooeokqo3weberqqc3okqoeorkcyyoes,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbeyokeokco3kcyokf3bodjcebekk3q3yqebekk3qb3ebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeekoqkwekcbo3kcyokf3ebeeekyorokceberkqekqf3eebercok3qebercoky3kckceebeebcekyoeoqeebecqok3qocqebeqoc3yqf3ebeec3eekyebe3eoebe9okeooebeooeebeoqooboeeoqebeoqooeokqo3webeoqkqodeeykrcckoqeebeokdyoqqoebe3wokdoqc3oqebeecweberqqc3okqoeeberqqc3okqoeorkcyyoes,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbeyokeokco3kcyokf3bodjcebekk3q3yqebekk3qb3ebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeekoqkwekcbo3kcyokf3ebeeekyorokceberqqc3okqoeeoeeberkqekqf3eebercoky3kckceebeebcekyoeoqeebecqok3qocqebeec3eekyebe3eoebe9okeooebeoqooc3k3qebeoqooboeeoqebeoqooeokqo3webeokdyoqqoebeqkcoeeekebe3wokdoqc3oqeberqqc3okqoees,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbeyokeokco3kcyokf3bodjcebekk3q3yqebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeeekyorokceberqqc3okqoeeoeeberkqekqf3eebeebcekyoeoqeebeqoc3yqf3ebeec3eekyebecyyorcoebeooeebeoqooc3k3qebeoqooboeeoqebeoqooeokqo3wes,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbeyokeokco3kcyokf3bodjcebekk3q3yqebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeeoqocyooqebeeekyorokcebekckc3rqokebekckbocqoyf3eberqqc3okqoeeoeeberkqekqf3eeberkqekkeebercok3qebercoky3kckceebeebcekyoeoqeebecqok3qocqebeec3eekyebe9okeooebeooeebeoqooc3k3qebeoqooboeeoqebeoqooeokqo3weberqqc3okqoees,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbobk3qeeoo3ekokcykorokbedocoqqqkqf3ebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeeedo3okeebeeekqf3eebeekyeoo3ekokcyko3kcyokf3bekckc3rqokebekckbocqoyf3eberkqekqf3eebeo3kcckdebe9okeooeborqkee3qeoycqkbebdoqeo3es,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbobk3qeeoo3ekokcykorokbedocoqqqkqf3ebeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeeedo3okeebeekyeoo3ekokcyko3kcyokf3bekckc3rqokebekckbocqoyf3eberkqekqf3eebeo3kcckdebe9okeooebeqc3ec3qeeborqkee3qeoycqkbeyk3qrkeeebebdoqeo3es,goyqoeoo3ekokcykbobk3qeeoo3ekokcykoqqqbobk3qeeoo3ekokcykorokbeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeeedo3okeeberkqekqf3eebeo3kcckdebe9okeooebeqc3ec3qeeborqkee3qeoycqkbeyk3qrkeees,goyqoeoo3ekokcykbobk3qeeoo3ekokcykorokbeq3ok3qrkeeebeqkqcre3qeokqo3wkqf3eebeeekqf3eeberkqekqf3eebeo3kcckdebe9okeooebeqc3ec3qeebeyk3qrkeees,gs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
train_df.drop(["txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)

****

In [62]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
estimator=RandomForestRegressor(max_depth=8)
mice = IterativeImputer(estimator=estimator,random_state=11,skip_complete=True)
imp_train_df=mice.fit_transform(train_df)
imp_train_df=pd.DataFrame(imp_train_df,columns=train_df.columns)

In [65]:
test_df

,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,categ_8,txt_2,categ_9,num_10,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,Date_year,Date_month,Date_day
0,2,a:o@`oq'<e@@@1!)@`RRK@w@Kqo@o:q`c,4,168982,3,0,49,1,<o<,74,20.0,11,2,<o<,2007.0,2368,<o<,2022,4,9
1,2,"a:o@eBcqKo""'@Z!?@`q]:@ZZu@:e""@,w1@o`K:R'",2,74500,3,0,19,39,gEo:q@`c<]:K:c<<'Eb@E`oY'qo@]'@q'`3+Eb@EO+3'Kc...,277,999.0,1,1,gEo:q@`c<]:K:c<<'Eb@E`oY'qo@]'@q'`3+Eb@EO+3'Kc...,NaN,2368,<o<,2021,5,11
2,2,"a:o@q:c@Bq'Y:3Y@Z!,+@Z)_",4,82996,3,0,34,33,<o<,65,999.0,1,1,<o<,NaN,2368,<o<,2021,5,12
3,2,a:o@B:`o<Kc@Z!)@;;`D@o`K:R'@uB@`+:Y@Yo<3@3eO@O...,4,36470,3,0,19,25,gEO+3'KccKDEb@Ecq]:<oK'3q@]'@Ocq]Es,261,999.0,1,1,gEO+3'KccKDEb@Ecq]:<oK'3q@]'@Ocq]Es,2016.0,2368,<o<,2021,5,11
4,2,"a:o@eBcqKo""'@1!)@`q]:@Z,)@,w1@.oB@3qOo<@q:]'q",2,185000,3,0,51,39,gEo:q@`c<]:K:c<<'Es,277,999.0,1,1,gEo:q@`c<]:K:c<<'Es,NaN,2368,<o<,2021,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2,"a:o@eKc<:`@Z!)@K@""]:@Z))`D@:e""@]'e:""<@'3qc;]@K",4,55711,3,0,34,41,gE`oY'qo@]'@q'`3+Eb@EO+3'KccKDEb@Ecq]:<oK'3q@]...,26,999.0,1,1,gE`oY'qo@]'@q'`3+Eb@EO+3'KccKDEb@Ecq]:<oK'3q@]...,2018.0,2368,<o<,2021,6,10
14996,2,"a:o@Bqc@`''E]@Z!;@K@""]:@1),`D@""K@]`K?",1,37580,3,0,49,4,<o<,72,999.0,1,1,<o<,2019.0,2368,<o<,2021,7,22
14997,2,a:o@B:`o<Kc@Z!)@K'`D,4,34846,3,0,49,25,<o<,261,589.0,10,1,<o<,2017.0,2368,<o<,2022,4,3
14998,2,"a:o@R'<""o@Z!,@`RRK@e3B'q@Bo`a@@u^!)))Y@<oR:""oK...",4,58700,3,0,46,42,"gE""BeEb@EO+3'KccKDEb@Eq'""3+oK'3q@]'@R:K'ee'Eb@...",41,307.0,10,1,"gE""BeEb@EO+3'KccKDEb@Eq'""3+oK'3q@]'@R:K'ee'Eb@...",2013.0,2368,<o<,2022,4,4


In [63]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
imp_test_df=mice.fit_transform(test_df)
imp_test_df=pd.DataFrame(imp_test_df,columns=test_df.columns)

ValueError: could not convert string to float: "a:o@`oq'<e@@@1!)@`RRK@w@Kqo@o:q`c"

In [ ]:
train_df = pd.concat([train_df, bow], axis=1)

In [ ]:
imp_train_df.isna().sum()

In [ ]:
y_train=imp_train_df["y"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(imp_train_df)
X_test = mms.transform(imp_test_df)


In [ ]:
y_train=np.array(y_train).reshape(-1,1)

In [ ]:
mms.fit_transform(y_train)

In [ ]:
X_test.shape, X_train.shape

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.1)

****

In [ ]:
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(min_samples_split=15)
Rf.fit(xtrain, ytrain)
ypred=Rf.predict(xtest)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(ytest, ypred)

In [ ]:
Rf.fit(xtest, ytest)

In [ ]:
# y_pred4 = Rf.predict(xtest) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
# submission.to_csv('submission4.csv', index=False)

****

In [ ]:
from sklearn import  ensemble
params = {
    "n_estimators": 500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(xtrain, ytrain)
mse = mean_squared_error(ytest, reg.predict(xtest))
mse

In [ ]:
# mreg.fit(X_train, y_train)
# y_pred4 = reg.predict(X_test) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'_id': Id,'y_pred': y_pred_original4})
# submission.to_csv('submission12.csv', index=False)

****

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model1 = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model1, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.5f (%.5f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model1 = LGBMRegressor()
model1.fit(xtrain, ytrain)
yhatt = model1.predict(xtest)
mean_squared_error(ytest, yhatt)

In [ ]:
# model1.fit(X_train, y_train)
# y_pred4 = model1.predict(X_test) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'_id	': Id,'y_pred': y_pred_original4})
# submission.to_csv('submission13.csv', index=False)